# b.     STORING THE DATA

#### ALL REQUIRED IMPORTS

In [275]:
import os
import csv
import requests
from pprint import pprint
import json
from datetime import datetime
import string
import glob
import shutil


In [2]:
directory = os.path.dirname(os.getcwd())
directory

'/Users/sonalichaudhari/Desktop'

In [117]:
key = os.environ.get('ZOM_KEY')
print(key)

2ce803cdd97585066baf42ed16933114


In [118]:
header1={'user-key':key}
header1

{'user-key': '2ce803cdd97585066baf42ed16933114'}

In [16]:
#id_list = []
#result=[]

In [18]:
path =  directory + '/FINAL_Python'

for subdir, dirs, files in os.walk(path):
    for file in files:
        if file == 'IDS.csv':
            reqpath = os.path.join(subdir,file)
            #print(reqpath)
            with open(reqpath, newline='') as f:
                reader = csv.reader(f)
                #reader.next()
                try:
                    for record in reader:
                        c = record[0]
                        #print(c)
                        if c not in id_list:
                            id_list.append(c)                       
                except csv.Error as e:
                    sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))


In [200]:
len(id_list)

1286

In [22]:
id_list = [int(x) for x in id_list]

#### COLLECTING RESTAURANT DATA PER CITY ID

In [119]:
#m=[]
for f in id_list[1001:1501]:
    try:
        v = str(f)
        url1 = 'https://developers.zomato.com/api/v2.1/search?entity_id='+v+'&entity_type=city&count=200'
        #result.append(requests.get(url1,headers=header1).json())
        m.append('1')
    except ValueError:
        continue

In [122]:
len(id_list)

1286

In [197]:
RES_ID = []
with open('RES_IDS.csv', 'w') as csvfile:    # Creating a CSV file named 'Data' in write mode as csvfile
    fieldnames = ['RES_ID']  #Naming the title Row of the CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for x in range(len(final)):
            r = final[x]['restaurant']['R']['res_id']
            if r not in RES_ID:
                RES_ID.append(r)
                writer.writerow({'RES_ID':r}) 
len(RES_ID)

23124

In [177]:
reqpath = path + '/Data'
reqpath

'/Users/sonalichaudhari/Desktop/FINAL_Python/Data'

In [435]:
COUNTRY={1:'India',14:'Australia',216:'United States',37:'Canada',123:'Malaysia',215:'United Kingdom',\
         54:'Czech Republic', 30: 'Brasil',42:'Chile',94:'Indonesia',97:'Ireland',99:'Italy',\
        112:'Lebanon',148:'New Zealand', 162:'Philippines',163:'Poland',164:'Portugal',166:'Qatar',\
        184:'Singapore',185:'Slovakia', 189:'South Africa',191:'Sri Lanka', 208:'Turkey',214:'UAE',}
with open('COUNTRY.json', 'w') as outfile:
    json.dump(COUNTRY, outfile)
len(COUNTRY)

24

In [346]:
#Storing
final=[]
for i in range(len(result)):
    for j in range(len(result[i]['restaurants'])):
        if result[i]['restaurants'][j] not in final:
            final.append(result[i]['restaurants'][j])
            
len(final)


23706

In [354]:
#APPENDING COUNTRY NAME TO EVERY RESTAURANT JSON FILE
for i in range(len(final)):
    countryid=final[i]['restaurant']['location']['country_id']
    if(countryid in COUNTRY.keys()):
        h=COUNTRY[countryid]
        final[i]['restaurant']['location']['country_name']=h


In [433]:
# LOADING THE JSON FILES IN TEMPORARY FOLDER
p = directory+'/FINAL_Python/TEMP/'
for y in final[20000:23706]:
    rest = y['restaurant']['name'].replace('/',' ')+'_'+str(y['restaurant']['R']['res_id'])
    with open(p + rest + str(datetime.now()) +'.json', 'w',encoding="utf-8") as outfile:
                        json.dump(y, outfile, ensure_ascii=False)
    

In [434]:
for json_file in glob.glob("/Users/sonalichaudhari/Desktop/FINAL_Python/TEMP/*.json"):
    file_distribution(read_files(json_file),json_file)

In [364]:
#final[1748]
#1836
#3026

In [382]:
final[1836]['restaurant']['R']['res_id']

16611484

In [296]:
def read_files(file):      # Create a function to read files. 
    with open (file, encoding='utf-8', errors="ignore") as f:   # Need to deal with unicode problem.
        content = json.load(f)
        hierarchy_list = {'country': content['restaurant']['location']['country_name'],\
                          'city': content['restaurant']['location']['city'],\
                          'name': content['restaurant']['name']}
        # Remove the ',' in some names for later output csv file.
        return hierarchy_list

In [298]:
def file_distribution(dict, file):   # Generate hierarchy folder structure and dump json file
    file_name = '/Users/sonalichaudhari/Desktop/FINAL_Python/Data/'\
    +'/' + dict['country'] + '/' + dict['city'] + '/' + dict['name'] \
                + '.json'
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    #fp = open (file_name, 'w')
    #with open (file, 'r', encoding='utf-8', errors="ignore") as jfile:
        #fp.write(json.dumps(json.load(jfile)))      # Try to find out the difference between dump and dumps
    shutil.move(file,file_name)